## Model Development & Performance

In [20]:
import pandas as pd
import numpy as np
import pickle
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

#### Data Loading

In [2]:
df = pd.read_csv('../data/modelingData/modelingDataFrame.csv')

We will start with classic models to evaluate their effectiveness. As a first step, we will attempt to predict the number of people without power. Initially, we will remove columns that are not relevant to our task, such as time-related data and precise location identifiers (e.g., county name and code). The remaining data will be transformed and prepared for modeling.

### XGBRegressor

In [4]:
event_names = ['Astronomical Low Tide', 'Extreme Cold/Wind Chill', 'Flood','Winter Weather', 
               'Wildfire', 'Heavy Rain', 'Cold/Wind Chill', 'Dense Fog', 'Frost/Freeze', 'Strong Wind',
               'Lake-Effect Snow', 'Funnel Cloud', 'Flash Flood', 'Heavy Snow', 'Ice Storm', 
               'Thunderstorm Wind', 'Avalanche', 'Excessive Heat', 'Coastal Flood', 'Storm Surge/Tide', 
               'Sleet', 'Debris Flow', 'Winter Storm', 'Tropical Storm', 'Dust Storm', 'Drought', 
               'Blizzard', 'Lightning', 'Tornado', 'Hail', 'Rip Current', 'Heat', 'Freezing Fog', 
               'High Surf', 'High Wind',]

In [5]:
target = 'CustomersOut'

numeric_features = ['Tmin', 'Tmax', 'Tavg', 'Ppt', 'Lat', 'Lng']
categorical_features = ['Season', 'Region', 'Division']
event_features = [col for col in df.columns if col in event_names]

X = df[numeric_features + categorical_features + event_features]
y = df[target]

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

event_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

# ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features),
    ('event', event_transformer, event_features)
])

In [8]:
X_train_proc = preprocessor.fit_transform(X_train)
X_val_proc = preprocessor.transform(X_val)

In [29]:
xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    verbosity=1,
    colsample_bytree=0.3,
    early_stopping_rounds=20,
    random_state=42
)

xgb.fit(
    X_train_proc, y_train,
    eval_set=[(X_val_proc, y_val)],
    verbose=True
)

[0]	validation_0-rmse:1479.44628
[1]	validation_0-rmse:1478.76089
[2]	validation_0-rmse:1478.22730
[3]	validation_0-rmse:1477.31792
[4]	validation_0-rmse:1476.28734
[5]	validation_0-rmse:1476.25554
[6]	validation_0-rmse:1474.07049
[7]	validation_0-rmse:1473.13567
[8]	validation_0-rmse:1472.70791
[9]	validation_0-rmse:1472.28972
[10]	validation_0-rmse:1471.00776
[11]	validation_0-rmse:1470.85316
[12]	validation_0-rmse:1469.18733
[13]	validation_0-rmse:1468.38280
[14]	validation_0-rmse:1468.30442
[15]	validation_0-rmse:1467.59923
[16]	validation_0-rmse:1465.09802
[17]	validation_0-rmse:1464.53772
[18]	validation_0-rmse:1464.02536
[19]	validation_0-rmse:1463.91608
[20]	validation_0-rmse:1463.74474
[21]	validation_0-rmse:1463.73358
[22]	validation_0-rmse:1462.90520
[23]	validation_0-rmse:1461.81601
[24]	validation_0-rmse:1461.37282
[25]	validation_0-rmse:1461.28740
[26]	validation_0-rmse:1461.27876
[27]	validation_0-rmse:1461.05672
[28]	validation_0-rmse:1461.04833
[29]	validation_0-rmse:1

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=20,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [12]:
# Model XGBoost
xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    verbosity=2
)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.001, 0.5),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=1,
    random_state=42
)

random_search.fit(X_train_proc, y_train)

with open("random_search_results.pkl", "wb") as f:
    pickle.dump(random_search, f)

print("✅ Zapisano RandomizedSearchCV do pliku random_search_results.pkl")
print("✅ Najlepsze parametry:", random_search.best_params_)
print("📉 Najlepszy neg-RMSE (cross-val):", random_search.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END colsample_bytree=0.8123620356542087, learning_rate=0.4763571532049581, max_depth=5, n_estimators=171, subsample=0.8795975452591109; total time=  54.1s
[CV] END colsample_bytree=0.8123620356542087, learning_rate=0.4763571532049581, max_depth=5, n_estimators=171, subsample=0.8795975452591109; total time=  49.4s
[CV] END colsample_bytree=0.8123620356542087, learning_rate=0.4763571532049581, max_depth=5, n_estimators=171, subsample=0.8795975452591109; total time=  44.3s
[CV] END colsample_bytree=0.7468055921327309, learning_rate=0.07899726016810132, max_depth=5, n_estimators=187, subsample=0.8001125833417065; total time=  56.7s
[CV] END colsample_bytree=0.7468055921327309, learning_rate=0.07899726016810132, max_depth=5, n_estimators=187, subsample=0.8001125833417065; total time=  52.6s
[CV] END colsample_bytree=0.7468055921327309, learning_rate=0.07899726016810132, max_depth=5, n_estimators=187, subsample=0.8001125833417

In [15]:
best_model = random_search.best_estimator_
joblib.dump(best_model, "../models/xgb_model.pkl")
print("✅ Zapisano najlepszy model XGBoost do best_xgb_model.pkl")

✅ Zapisano najlepszy model XGBoost do best_xgb_model.pkl


In [19]:
y_pred = best_model.predict(X_val_proc)

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

RMSE: 1414.8726
MAE: 91.6789
R²: 0.0882
